# Business Intelligence exam project march 2024
Group members: Rasmus Arendt, Deniz Denson, Victor Christensen & Marcus Løbel

# Imports

In [ ]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn import preprocessing as prep
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Load and clean the data

### We engineer & wrangle

In [ ]:
# Load the data. The data is from the Our World in Data's github (https://github.com/owid/covid-19-data/tree/master/public/data). downloaded on 10/03/2024
df = pd.read_csv("./Data/owid-covid-data.csv")

In [ ]:
#Check that the data is loaded correctly
df.sample(5)

In [ ]:
#Print the columns to see what we have, and later decide what to use
df.columns

### For our three hypothesis', we will use the following columns:

In [ ]:
columns_to_keep_hypo1 = ['iso_code','location', 'total_cases', 'gdp_per_capita','date']
columns_to_keep_hypo2 = ['iso_code', 'location', 'total_cases', 'date', 'total_vaccinations_per_hundred', 'population_density']
columns_to_keep_hypo3 = ['stringency_index', 'human_development_index' ]

### Creating a new dataframe with the columns we've chosen

In [ ]:
# new df
data_hypo1 = df[columns_to_keep_hypo1]
data_hypo2 = df[columns_to_keep_hypo2]

Check the data to see if it looks good

In [ ]:
#print a sample
print(data_hypo1.sample(5))
print(data_hypo2.sample(5))

Get the percentage of missing values in data_hypo1

In [ ]:
#Get percentage of missing values
missing_values = (data_hypo1.isnull().sum()/data_hypo1.shape[0])*100
missing_values

Remove the rows with missing values in the total_cases column, because we can't get that data from anywhere else and then check the missing values again. 

In [ ]:
# Remove rows
data_hypo1 = data_hypo1.dropna(subset=['total_cases'])
data_hypo2 = data_hypo2.dropna(subset=['total_cases'])

missing_values_hypo1 = (data_hypo1.isnull().sum()/data_hypo1.shape[0])*100
missing_values

Turn the date column into a datetime object, as that's more useful for us

In [ ]:
# Date column -> datetime
data_hypo1['date'] = pd.to_datetime(data_hypo1['date'])
data_hypo2['date'] = pd.to_datetime(data_hypo2['date'])

#Get a list of the iso codes, to see what countries we have data for

In [ ]:
# list of the iso codes
iso_codes_hypo1 = data_hypo1['iso_code'].unique()
iso_codes_hypo1

Get a list of owid special codes (which are not countries!)

In [ ]:
# owid special codes
iso_codes_owid = data_hypo1[data_hypo1['iso_code'].str.contains('OWID')]['iso_code'].unique()
iso_codes_owid

Remove the owid rows from the data, because we are only interested in actual countries

In [ ]:
#Removing rows
data_hypo1 = data_hypo1[~data_hypo1['iso_code'].str.contains('OWID')]
data_hypo2 = data_hypo2[~data_hypo2['iso_code'].str.contains('OWID')]

Check the data to make sure all the owid rows are removed

In [ ]:
# Check to see if it's done correct
iso_codes_owid = data_hypo1[data_hypo1['iso_code'].str.contains('OWID')]['iso_code'].unique()
iso_codes_owid

### We then find any countries that doesnt have any data in the population_density column.

Note: We use the iso code, because it is unique for each country, and other data might have different names for the same country or capital letters for a country 

In [ ]:
missing_population_density = data_hypo2[data_hypo2['population_density'].isnull()]['iso_code'].unique()
missing_population_density

We then load another dataset so we can fill some of the missing data in the population density column

In [ ]:
#load dataset
pop_density = pd.read_csv("./Data/population-density.csv")

We then find the first year present in the covid dataset

In [ ]:
first_year = data_hypo2['date'].min().year
first_year

And then the last year in the covid dataset

In [ ]:
last_year = data_hypo2['date'].max().year
last_year

.. because we can then remove rows that are not within the range of the first and last year of the covid dataset, because we only need data from inside the covid-period. 

In [ ]:

#discard unusuable years
pop_density = pop_density[pop_density['Year'] >= first_year]
pop_density = pop_density[pop_density['Year'] <= last_year]

Check the data to see its size

In [ ]:
pop_density.shape

And if it looks good :)

In [ ]:
pop_density.sample(5)

Get the percentage of missing values in the pop_density

In [ ]:
(pop_density.isnull().sum()/pop_density.shape[0])*100

Check which rows has missing values in the Code column

In [ ]:
pop_density[pop_density['Code'].isnull()]

We then remove rows with missing values in the Code column, as they are not useful for our analysis, as they are not associated with any country, but rather a region, continent or group of people


In [ ]:
pop_density = pop_density.dropna(subset=['Code'])

We then re-check the percentage of missing values, to make sure the rows are correctly removed

In [ ]:
(pop_density.isnull().sum()/pop_density.shape[0])*100

Get the amount of countries in the population density dataset

In [ ]:
len(pop_density['Entity'].unique())

Rename the population density column to make it easier to work with, when we merge the datasets

In [ ]:

pop_density.rename(columns={'Entity':'location', 'Code':'iso_code','Year':'year', 'Population density': 'population_density'}, inplace=True)

Check the columns to see if the renaming was successful

In [ ]:
# Check if done correct
pop_density.sample(5)

Find out if the countries in the covid dataset, with missing population_density, are in the population density dataset

In [ ]:
# Discover if it exists
doesnt_exists = []
for code in missing_population_density:
    if not code in pop_density['iso_code'].unique():
        doesnt_exists.append(code)

print(len(doesnt_exists), len(missing_population_density))
print(doesnt_exists)
# this means that there is 5 countries in the covid dataset, with missing population_density, that are not in the population density dataset

Remove the countries that are not in the population density dataset from the covid dataset

In [ ]:
#remove excess countries that are not there
data_hypo2 = data_hypo2[~data_hypo2['iso_code'].isin(doesnt_exists)]

In [ ]:
data_hypo2.columns

### Next we will fill in the missing population density data, by using the population density dataset and a new dataframe as an intermediary dataframe

### This is done by finding the population density data for the countries with missing population density, and then putting this data into a new dataframe

In [ ]:
rows_with_missing_pop_density = data_hypo2[data_hypo2['population_density'].isnull()]
df_with_pop_filled = pd.DataFrame(columns=data_hypo2.columns)
for row in rows_with_missing_pop_density.iterrows():
    index = row[0]
    row = row[1]
    year = row['date'].year
    location = row['location']
    iso_code = row['iso_code']
    year_condition = pop_density['year'] == year
    iso_code_condition = pop_density['iso_code'] == iso_code
    combined_condition = year_condition & iso_code_condition
    pop_density_row = pop_density[combined_condition]
    #print(row)
    df_with_pop_filled.loc[index] = [iso_code, location, row['total_cases'], pop_density_row['population_density'].values[0], row['date'], row['total_vaccinations_per_hundred']]


Put the data from the intermediary dataframe into the original covid dataframe at the correct index

In [ ]:
# put the data from intermediary df to original df
data_hypo2['population_density'] = data_hypo2['population_density'].fillna(df_with_pop_filled['population_density'])

Drop the rows with missing values in the gdp_per_capita column, because we can't get that data from anywhere else that is up to date

In [ ]:
#dropping rows with missing values in gdp_per_capital column
data_hypo1 = data_hypo1.dropna(subset=['gdp_per_capita'])

In [ ]:
#dropping rows with missing values in population_density
data_hypo2 = data_hypo2.dropna(subset=['population_density'])

In [ ]:
(data_hypo2.isnull().sum()/data_hypo2.shape[0])*100

Load another dataset to so we can fill some of the missing data in the total_vaccinations_per_hundred column

In [ ]:
#load another dataset to fill data
vacc_per_hundred_dataset = pd.read_csv("./Data/covid-vaccination-doses-per-capita.csv")

In [ ]:
vacc_per_hundred_dataset.columns

In [ ]:
vacc_per_hundred_dataset.rename(columns={'Entity':'location', 'Code':'iso_code','Day':'date'}, inplace=True)

In [ ]:
vacc_per_hundred_dataset.shape

In [ ]:
rows_with_missing_vacc_per_hundred = data_hypo2[data_hypo2['total_vaccinations_per_hundred'].isnull()]
print(f"covid data is missing {len(rows_with_missing_vacc_per_hundred)} rows")

In [ ]:
# get the percentage of missing values
missing_values = (data_hypo2.isnull().sum()/data_hypo2.shape[0])*100
missing_values

In [ ]:
data_hypo2['date'].sort_values()

In [ ]:
vacc_per_hundred_dataset['date'].sort_values()

In [ ]:
vacc_per_hundred_dataset['date'] = pd.to_datetime(vacc_per_hundred_dataset['date'])

In [ ]:
def vacc_merge_datasets(dataset1, dataset2):
    # Convert dates to datetime objects
    dataset1['date'] = pd.to_datetime(dataset1['date'])
    dataset2['date'] = pd.to_datetime(dataset2['date'])

    # Merge datasets based on 'iso_code' and 'date'
    merged = pd.merge(dataset1, dataset2, on=['iso_code', 'date'], how='left', suffixes=('_1', '_2'))

    # Replace missing values in 'total_vaccinations_per_hundred_1' with values from 'total_vaccinations_per_hundred_2'
    merged['total_vaccinations_per_hundred'] = merged['total_vaccinations_per_hundred_1'].fillna(merged['total_vaccinations_per_hundred_2'])

    # Drop unnecessary columns
    merged.drop(['total_vaccinations_per_hundred_1', 'total_vaccinations_per_hundred_2'], axis=1, inplace=True)

    # Fill missing values in 'total_vaccinations_per_hundred' with most recent values from dataset2
    merged['total_vaccinations_per_hundred'].fillna(method='ffill', inplace=True)

    return merged

In [ ]:
# test_dataset = fill_missing_vaccination_data(data_hypo2, vacc_per_hundred_dataset)
intermediary_dataset = vacc_merge_datasets(data_hypo2, vacc_per_hundred_dataset)


In [ ]:
intermediary_dataset.sample(5)

Put the data from the intermediary dataframe into the original covid dataframe at the correct index

In [ ]:
data_hypo2['total_vaccinations_per_hundred'] = data_hypo2['total_vaccinations_per_hundred'].fillna(intermediary_dataset['total_vaccinations_per_hundred'])

Remove the rows with missing values in the total_vaccinations_per_hundred column, because we can't get that data from anywhere else and check the missing values again

In [ ]:

data_hypo2 = data_hypo2.dropna(subset=['total_vaccinations_per_hundred'])

In [ ]:
data_hypo2.isnull().sum()/data_hypo2.shape[0]*100

# Hypothesis 1

### "We do not believe that there is a correlation between the number of infected individuals in relation to a country's Gross National Product (GNP) per capita."

Copy the covid data to use for hypothesis 1

In [ ]:
# Copy data
data_hypothesis_1 = data_hypo1[['location', 'total_cases', 'gdp_per_capita', 'date']]

Get an overview of the data

In [ ]:
data_hypothesis_1.describe()

Get the last row for each countries latest observation

In [ ]:

last_row = data_hypothesis_1.groupby('location').last().reset_index()
last_row.sample(5)

Check if the last row for each country has the same date, so the data fits

In [ ]:

last_row['date'].max() == last_row['date'].min()

We then create a subset of the data to use for the graph, where it is sorted by the total_cases

In [ ]:
# graph of the cumulative cases per country
data_hypothesis_1_subset = last_row.sort_values('total_cases', ascending=False)
plt.figure(figsize=(200,100))
sns.barplot(x='location', y='total_cases', data=data_hypothesis_1_subset)
plt.xticks(rotation=90)
plt.show()

Scatterplot to show gdp_per_capita as x and total_cases for each country as y

In [ ]:
# scatterplot
plt.figure(figsize=(10, 6))
sns.scatterplot(x='gdp_per_capita', y='total_cases', data=last_row)
plt.title('Total cases vs GDP per capita')
plt.show()

Here we can see the each country's BNP per capita and the number of infected people in the country. We can also see that there is 3 outliers in the data. These are (China), (USA) and (Italy). We will remove these from the data to get a better overview of the data.

In [ ]:
data_hypo1['date'] = pd.to_datetime(data_hypo1['date'])

In [ ]:
years = data_hypo1['date'].dt.year.unique()
years

Piechart to show  the top 5 countries with the highest average total cases for each year that we have data for as a single graph.

In [ ]:

fig = plt.figure(figsize=(20, 20))

for i, y in enumerate(years, start=1):
    plt.subplot(3, 2, i)
     # get the avg total cases per country for the year
    avg_total_cases = data_hypo1[data_hypo1['date'].dt.year == y].groupby('location')['total_cases'].mean()
    avg_total_cases = avg_total_cases.reset_index()
    avg_total_cases = avg_total_cases.sort_values(by='total_cases', ascending=False)
    # get the top 5 countries
    top_5 = avg_total_cases.head(5)
    plt.pie(top_5['total_cases'], labels=top_5['location'], autopct='%.1f%%',
            startangle=90, shadow=True)
    plt.title(y, fontsize=15)

plt.suptitle('Top 5 highest average total cases per country foreach year', fontsize=20)
plt.show()

# Hypothesis 2

### "We believe there is a connection between a country's population density and the number of COVID-19 cases, where higher population density correlates with more COVID-19 cases. That is to say, countries with more cases also had higher vaccination coverage."

Copy the necessary columns to the hypothesis 2 dataframe

In [ ]:
# Copy columns
data_hypothesis_2 = data_hypo2[['location', 'total_cases', 'total_vaccinations_per_hundred']]


# Check the data to see if it looks good
print(data_hypothesis_2.head())



In [ ]:
data_hypothesis_2.isnull().sum()

Remove rows with missing values in the 'total_cases' column

In [ ]:
data_hypothesis_2.dropna(subset=['total_cases'], inplace=True)


In [ ]:
data_hypothesis_2.describe()

Get the last row for each countries latest observation

In [ ]:
last_row = data_hypothesis_2.groupby('location').last().reset_index()
last_row.sample(5)

### Bar plot to show how vaccines per hundred people per country

In [ ]:

import plotly.express as px

# Sort the data by 'total_vaccinations_per_hundred' column
data_hypothesis_2_subset = data_hypothesis_2.sort_values('total_vaccinations_per_hundred', ascending=False)

# Create a bar plot with plotly
fig = px.bar(data_hypothesis_2_subset, x='location', y='total_vaccinations_per_hundred',
             labels={'total_vaccinations_per_hundred': 'Total Vaccinations per Hundred'},
             title='Total Vaccinations per Hundred by Country')

# Add hover data to show country names
fig.update_traces(hovertemplate='<b>%{x}</b><br>Total Vaccinations per Hundred: %{y}')

# Rotate x-axis labels for better readability
fig.update_layout(xaxis_tickangle=-45)

# Show the plot
fig.show()


### Scatterplot to show how vaccines per hundred people per country

In [ ]:

# Create a scatter plot with plotly
fig = px.scatter(data_hypothesis_2_subset, x='total_cases', y='total_vaccinations_per_hundred', hover_name='location',
                 labels={'total_cases': 'Total COVID-19 Cases', 'total_vaccinations_per_hundred': 'Vaccinations per Hundred'},
                 title='Relationship between COVID-19 Cases and Vaccination Coverage')

# Show the plot
fig.show()



In [ ]:
# Find the top 5 countries with the highest average number of cases for vaccination coverage per hundred
top_5_countries = data_hypothesis_2_subset.groupby('location')['total_vaccinations_per_hundred'].mean().nlargest(5).index

# Create a subset of data containing only the top 5 countries
top_5_data = data_hypothesis_2_subset[data_hypothesis_2_subset['location'].isin(top_5_countries)]

plt.figure(figsize=(10, 6))
sns.barplot(x='location', y='total_vaccinations_per_hundred', data=top_5_data, order=top_5_countries)
plt.title('Top 5 lande med det højeste gennemsnitlige antal sager for vaccinationsdækning per hundrede')
plt.xlabel('Land')
plt.ylabel('Gennemsnitlig vaccinationsdækning per hundrede')
plt.xticks(rotation=45)
plt.show()


# Hypotese 3

### "We do not believe all countries are equally exposed to infection"
